In [ ]:
%matplotlib inline
import os
import shutil
import subprocess as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
import platform
if 'window' in platform.platform().lower():
    ppp = 'pest++'
    pref = ''
else:
    ppp = './pestpp'
    pref = './'

In [ ]:
base_dir = os.path.join("..","..","models","Freyberg","Freyberg_K_and_R")
[shutil.copy(os.path.join(base_dir,f),f) for f in os.listdir(base_dir)]
if not os.path.exists('base_resp_dir'):
    os.mkdir('base_resp_dir')
[shutil.copy2(os.path.join(base_dir,cf),os.path.join('base_resp_dir',cf)) for cf in os.listdir(base_dir)];


## First read in the PST file and find what are the starting values for K and R

In [ ]:
inpst = pyemu.Pst('freyberg.pst')
inpst.parameter_data

In [ ]:
os.system('{0} freyberg.pst'.format(ppp))

In [ ]:
inpst.phi_components

## Now make a CSV file of values to evaluate

In [ ]:
nsamples = 20
hk_start = inpst.parameter_data.loc[inpst.parameter_data.parnme=='hk1'].parval1.values[0]
hk_lower=3
hk_upper = 12
rch1_start = inpst.parameter_data.loc[inpst.parameter_data.parnme=='rch1'].parval1.values[0]
rch1_lower= 0.0001
rch1_upper= 0.00017
rch2_start = inpst.parameter_data.loc[inpst.parameter_data.parnme=='rch2'].parval1.values[0]


In [ ]:
hk_values = np.linspace(hk_lower,hk_upper,nsamples)
rch1_values = np.linspace(rch1_lower,rch1_upper,nsamples)
c = 0
with open("grid.csv",'w') as f:
    f.write("run_id,hk1,rch1,rch2\n")
    for v1 in hk_values:
        for v2 in rch1_values:
            f.write("{0:d},{1:15.6E},{2:15.6E},{3:15.6E}\n".format(c,v1,v2,rch2_start))
            c += 1
shutil.copy2('grid.csv','base_resp_dir')

In [ ]:
def resp_plot():
    df_in = pd.read_csv("grid.csv")
    df_out = pd.read_csv("sweep_out.csv", na_values='')
    resp_surf = np.zeros((nsamples,nsamples))
    c = 0
    for i,v1 in enumerate(hk_values):
        for j,v2 in enumerate(rch1_values):
            resp_surf[j,i] = df_out.loc[c,"phi"]
            c += 1
    fig = plt.figure(figsize=(5,5))
    ax = plt.subplot(111)
    X,Y = np.meshgrid(hk_values,rch1_values)
    # knock out extreme high values
    resp_surf = np.flipud(resp_surf)
    resp_surf[resp_surf>np.nanpercentile(resp_surf,75)] = np.nan
    resp_surf = np.ma.masked_where(np.isnan(resp_surf),resp_surf)
    p = ax.pcolor(X,Y,resp_surf,alpha=0.5,cmap="spectral")
    plt.colorbar(p)
    c = ax.contour(X,Y,resp_surf,levels=np.linspace(np.min(resp_surf),np.max(resp_surf),6),colors='k')
    plt.clabel(c)
    ax.set_xlim(hk_values.min(),hk_values.max())
    ax.set_ylim(rch1_values.min(),rch1_values.max())
    ax.set_xlabel("hk1 ($\\frac{L}{T}$)")
    ax.set_ylabel("rch1 ($L$)")

In [ ]:
pyemu.utils.start_slaves('base_resp_dir',"{0}sweep".format(pref),"freyberg.pst",20,slave_root='.',port=4004,master_dir='.')

In [ ]:
resp_plot()

### Now let's run PEST++ and see what we get for an estimate of the parameters
 * set NOPTMAX=15
 * run PEST++

In [ ]:
inpst = pyemu.Pst('freyberg.pst')
inpst.control_data.noptmax=-1
inpst.write('freybergk.pst')

In [ ]:
os.system('{0} freybergk.pst'.format(ppp))

### We can look at the final parameter values and their correlation

In [ ]:
pars = pd.read_csv('freybergk.ipar')
pars

In [ ]:
sc = pyemu.Schur('freybergk.jcb')
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

# What if we also have a calibration flux observation?

We can set this up by simply assigning non-zero weight to the observation called `rivflux_cal`

We've already read in the PST file, now we can change just that one value and write out a new PST file.

In [ ]:
# using the pandas set_value method
pst0p1 = pyemu.Pst('freyberg.pst')
pst0p1.observation_data.loc['rivflux_cal','weight']=0.1
pst0p1.write('freyberg_w_flux0p1.pst')
shutil.copy2('freyberg_w_flux0p1.pst',os.path.join('base_resp_dir','freyberg_w_flux0p1.pst'))

In [ ]:
os.system('{0} freyberg_w_flux0p1.pst'.format(ppp))

In [ ]:
pst0p1.phi_components

In [ ]:
pyemu.utils.start_slaves('base_resp_dir',"{0}sweep".format(pref),"freyberg_w_flux0p1.pst",20,slave_root='.',port=4004,master_dir='.')

In [ ]:
resp_plot()

### Now let's run PEST++ and see what we get for an estimate of the parameters
 * set NOPTMAX=-1
 * run PEST++

In [ ]:
pst0p1.control_data.noptmax=-1
pst0p1.write('freyberg_w_flux0p1.pst')
shutil.copy2('freyberg_w_flux0p1.pst',os.path.join('base_resp_dir','freyberg_w_flux0p1.pst'))

In [ ]:
os.system('{0} freyberg_w_flux0p1.pst'.format(ppp))

### We can look at the final parameter values and their correlation

In [ ]:
pars = pd.read_csv('freyberg_w_flux0p1.ipar')
pars

In [ ]:
sc = pyemu.Schur('freyberg_w_flux0p1.jcb')
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

In [ ]:
# using the pandas set_value method
pst0p001=pyemu.Pst('freyberg.pst')
pst0p001.observation_data.loc['rivflux_cal','weight']=0.001
pst0p001.write('freyberg_w_flux0p001.pst')
shutil.copy2('freyberg_w_flux0p001.pst',os.path.join('base_resp_dir','freyberg_w_flux0p001.pst'))

In [ ]:
os.system('{0} freyberg_w_flux0p001.pst'.format(ppp))

In [ ]:
pst0p001.phi_components

In [ ]:
pyemu.utils.start_slaves('base_resp_dir',"{0}sweep".format(pref),"freyberg_w_flux0p001.pst",20,slave_root='.',port=4004,master_dir='.')

In [ ]:
resp_plot()

In [ ]:
pst0p001.control_data.noptmax=-1
pst0p001.write('freyberg_w_flux0p001.pst')

In [ ]:
os.system('{0} freyberg_w_flux0p001.pst'.format(ppp))

In [ ]:
pars = pd.read_csv('freyberg_w_flux0p001.ipar')
pars

In [ ]:
sc = pyemu.Schur('freyberg_w_flux0p001.jcb')
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

In [ ]:
# using the pandas set_value method
pst0p01=pyemu.Pst('freyberg.pst')
pst0p01.observation_data.loc['rivflux_cal','weight']=0.01
pst0p01.write('freyberg_w_flux0p01.pst')
shutil.copy2('freyberg_w_flux0p01.pst',os.path.join('base_resp_dir','freyberg_w_flux0p01.pst'))

In [ ]:
os.system('{0} freyberg_w_flux0p01.pst'.format(ppp))

In [ ]:
pst0p01.phi_components

In [ ]:
pyemu.utils.start_slaves('base_resp_dir',"{0}sweep".format(pref),"freyberg_w_flux0p01.pst",20,slave_root='.',port=4004,master_dir='.')

In [ ]:
resp_plot()

In [ ]:
pst0p01.control_data.noptmax=-1
pst0p01.write('freyberg_w_flux0p01.pst')
shutil.copy2('freyberg_w_flux0p01.pst',os.path.join('base_resp_dir','freyberg_w_flux0p01.pst'))

In [ ]:
os.system('{0} freyberg_w_flux0p01.pst'.format(ppp))

In [ ]:
pars = pd.read_csv('freyberg_w_flux0p01.ipar')
pars

In [ ]:
sc = pyemu.Schur('freyberg_w_flux0p01.jcb')
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

In [ ]:
pst0p01.control_data.noptmax=15
pst0p01.write('freyberg_w_flux0p01.pst')

In [ ]:
os.system('{0} freyberg_w_flux0p01.pst'.format(ppp))

In [ ]:
sc = pyemu.Schur('freyberg_w_flux0p01.jcb')
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()